In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import numpy as np
import pandas as pd
from bayesian_testing.experiments import BinaryDataTest, DeltaLognormalDataTest

In [2]:
df = pd.read_csv("data/session_data.csv")

In [3]:
# example session data - each row represent one session
len(df)
df.head()

94500

,conversion,date,revenue,source,variant
0,0,2021-08-07,0.000000,desktop,B
1,1,2021-08-05,7.241015,desktop,C
2,0,2021-08-06,0.000000,desktop,A
3,0,2021-08-05,0.000000,desktop,C
4,0,2021-08-03,0.000000,desktop,A


In [4]:
# summary statistics per variant

summary = df.groupby('variant')[['variant', 'conversion', 'revenue']]\
            .agg({'variant': 'count', 'conversion': 'sum','revenue': 'sum'})\
            .rename(columns = {'variant': 'sessions', 'conversion': 'conversions'})

summary['conversion_rate'] = summary['conversions'] / summary['sessions']
summary['revenue_per_session'] = summary['revenue'] / summary['sessions']
summary['revenue_per_converted_sessions'] = summary['revenue'] / summary['conversions']

summary

,sessions,conversions,revenue,conversion_rate,revenue_per_session,revenue_per_converted_sessions
variant,,,,,,
A,31500,1580,30830.025613,0.050159,0.978731,19.512674
B,32000,1700,35203.216888,0.053125,1.100101,20.707775
C,31000,1550,37259.563364,0.050000,1.201921,24.038428


# Conversion Test

In [5]:
conv_test = BinaryDataTest()

In [6]:
variant_A = df['conversion'][(df.variant == 'A')].values
variant_B = df['conversion'][(df.variant == 'B')].values
variant_C = df['conversion'][(df.variant == 'C')].values

variant_A
max(variant_A)

array([0, 0, 0, ..., 0, 0, 0])

1

In [7]:
conv_test.add_variant_data("A", variant_A)

In [8]:
conv_test.add_variant_data("B", variant_B)

In [9]:
conv_test.add_variant_data_agg("C", len(variant_C), sum(variant_C))

In [10]:
conv_test.data
conv_test.variant_names
conv_test.totals
conv_test.positives

{'A': {'totals': 31500, 'positives': 1580, 'a_prior': 0.5, 'b_prior': 0.5},
 'B': {'totals': 32000, 'positives': 1700, 'a_prior': 0.5, 'b_prior': 0.5},
 'C': {'totals': 31000, 'positives': 1550, 'a_prior': 0.5, 'b_prior': 0.5}}

['A', 'B', 'C']

[31500, 32000, 31000]

[1580, 1700, 1550]

In [11]:
conv_test.probabs_of_being_best(sim_count = 20000, seed=52)

{'A': 0.04185, 'B': 0.92235, 'C': 0.0358}

In [12]:
conv_test.expected_loss(sim_count = 20000, seed=52)

{'A': 0.0030138, 'B': 6.06e-05, 'C': 0.0031649}

In [13]:
results = conv_test.evaluate()
print(pd.DataFrame(results).to_markdown(tablefmt="grid", index=False))

+-----------+----------+-------------+-----------------+------------------+------------------------+-------------------+-----------------+
| variant   |   totals |   positives |   positive_rate |   posterior_mean | credible_interval      |   prob_being_best |   expected_loss |
+===========+==========+=============+=================+==================+========================+===================+=================+
| A         |    31500 |        1580 |         0.05016 |          0.05017 | [0.047736, 0.0525938]  |            0.0408 |       0.0030418 |
+-----------+----------+-------------+-----------------+------------------+------------------------+-------------------+-----------------+
| B         |    32000 |        1700 |         0.05312 |          0.05314 | [0.0507408, 0.0556564] |            0.928  |       5.23e-05  |
+-----------+----------+-------------+-----------------+------------------+------------------------+-------------------+-----------------+
| C         |    31000 |   

In [14]:
#reversed_probabilities (for case when "best = min")
conv_test.probabs_of_being_best(sim_count = 20000, seed=52, min_is_best=True)
conv_test.expected_loss(sim_count = 20000, seed=52, min_is_best=True)
conv_test.credible_intervals(sim_count = 20000, seed=52, min_is_best=True)

{'A': 0.4594, 'B': 0.00925, 'C': 0.53135}

{'A': 0.000781, 'B': 0.0037342, 'C': 0.0006299}

{'A': [0.0477826, 0.0526302],
 'B': [0.0506933, 0.0555936],
 'C': [0.0476604, 0.0524757]}

# Revenue Test

In [15]:
rev_test = DeltaLognormalDataTest()

In [16]:
variant_A = df['revenue'][(df.variant == 'A')].values
variant_B = df['revenue'][(df.variant == 'B')].values
variant_C = df['revenue'][(df.variant == 'C')].values

variant_A
max(variant_A)

array([0., 0., 0., ..., 0., 0., 0.])

318.85100060418284

In [17]:
rev_test.add_variant_data("A", variant_A)
rev_test.add_variant_data("B", variant_B)

In [18]:
# adding variant using aggregated data
rev_test.add_variant_data_agg(
    name="C",
    totals=len(variant_C),
    positives=sum(x > 0 for x in variant_C),
    sum_values=sum(variant_C),
    sum_logs=sum([np.log(x) for x in variant_C if x > 0]),
    sum_logs_2=sum([np.square(np.log(x)) for x in variant_C if x > 0]),
)

In [19]:
# rev_test.data
rev_test.variant_names
rev_test.totals
rev_test.positives
rev_test.sum_values
rev_test.sum_logs
rev_test.sum_logs_2

['A', 'B', 'C']

[31500, 32000, 31000]

[1580, 1700, 1550]

[30830.02561293968, 35203.216888281, 37259.56336401027]

[3831.806394737818, 4211.729867679853, 4055.9652348481686]

[11029.923165846512, 12259.518683969136, 12357.91186291399]

In [20]:
rev_test.probabs_of_being_best()

{'A': 0.0002, 'B': 0.03315, 'C': 0.96665}

In [21]:
rev_test.expected_loss()

{'A': 0.2206517, 'B': 0.1204936, 'C': 0.0008737}

In [22]:
rev_test.credible_intervals()

{'A': [0.9083588, 1.0643646],
 'B': [1.0042736, 1.1693412],
 'C': [1.1076003, 1.306942]}

In [23]:
results = rev_test.evaluate()
print(pd.DataFrame(results).to_markdown(tablefmt="grid", index=False))

+-----------+----------+-------------+--------------+--------------+-----------------------+------------------+------------------------+-------------------+-----------------+
| variant   |   totals |   positives |   sum_values |   avg_values |   avg_positive_values |   posterior_mean | credible_interval      |   prob_being_best |   expected_loss |
+===========+==========+=============+==============+==============+=======================+==================+========================+===================+=================+
| A         |    31500 |        1580 |      30830   |      0.97873 |               19.5127 |          0.98278 | [0.9081132, 1.0651011] |           0.00015 |        0.219287 |
+-----------+----------+-------------+--------------+--------------+-----------------------+------------------+------------------------+-------------------+-----------------+
| B         |    32000 |        1700 |      35203.2 |      1.1001  |               20.7078 |          1.08266 | [1.0045567, 1

# Probabilities in time

In [24]:
# using time data we can generate probabilities of being best over time (cumulatively)

dates = np.sort(df.date.unique())
pbbs_conv_in_time = []
pbbs_rev_in_time = []
for i in dates:
    variant_A_conv = df[(df.variant == 'A') & (df.date <= i)]['conversion'].values
    variant_B_conv = df[(df.variant == 'B') & (df.date <= i)]['conversion'].values
    variant_C_conv = df[(df.variant == 'C') & (df.date <= i)]['conversion'].values
    
    variant_A_rev = df[(df.variant == 'A') & (df.date <= i)]['revenue'].values
    variant_B_rev = df[(df.variant == 'B') & (df.date <= i)]['revenue'].values
    variant_C_rev = df[(df.variant == 'C') & (df.date <= i)]['revenue'].values

    conv_test = BinaryDataTest()
    conv_test.add_variant_data("A", variant_A_conv)
    conv_test.add_variant_data("B", variant_B_conv)
    conv_test.add_variant_data("C", variant_C_conv)
    
    rev_test = DeltaLognormalDataTest()
    rev_test.add_variant_data("A", variant_A_rev)
    rev_test.add_variant_data("B", variant_B_rev)
    rev_test.add_variant_data("C", variant_C_rev)
    
    pbbs_conv_in_time.append({i: conv_test.probabs_of_being_best()})
    pbbs_rev_in_time.append({i: rev_test.probabs_of_being_best()})

In [25]:
pbbs_conv_in_time

[{'2021-08-01': {'A': 0.70065, 'B': 0.17965, 'C': 0.1197}},
 {'2021-08-02': {'A': 0.37235, 'B': 0.2207, 'C': 0.40695}},
 {'2021-08-03': {'A': 0.1893, 'B': 0.65485, 'C': 0.15585}},
 {'2021-08-04': {'A': 0.16005, 'B': 0.7431, 'C': 0.09685}},
 {'2021-08-05': {'A': 0.03215, 'B': 0.869, 'C': 0.09885}},
 {'2021-08-06': {'A': 0.01375, 'B': 0.9705, 'C': 0.01575}},
 {'2021-08-07': {'A': 0.0419, 'B': 0.9227, 'C': 0.0354}}]

In [26]:
pbbs_rev_in_time

[{'2021-08-01': {'A': 0.0209, 'B': 0.27195, 'C': 0.70715}},
 {'2021-08-02': {'A': 0.0584, 'B': 0.2102, 'C': 0.7314}},
 {'2021-08-03': {'A': 0.0648, 'B': 0.2057, 'C': 0.7295}},
 {'2021-08-04': {'A': 0.02095, 'B': 0.1761, 'C': 0.80295}},
 {'2021-08-05': {'A': 0.00065, 'B': 0.04725, 'C': 0.9521}},
 {'2021-08-06': {'A': 0.0004, 'B': 0.05005, 'C': 0.94955}},
 {'2021-08-07': {'A': 0.0002, 'B': 0.03445, 'C': 0.96535}}]